# 系统发育推断

## 准备工作,给每个基因文件的标识符信息添加对应的物种名信息

In [43]:
! pwd
#调用脚本，内有注释，完成标识信息的添加
! bash ./src/modified_ID.sh data/ modified_data/ fna

/home/liuji/project/evolution-tutorials


## 1、选取合适的分子标记进行多重比对、修剪、构树，本教程以直系同源单拷贝基因为例

In [44]:
#根据基因名合并物种序列,准备构建基因树
! pwd
! python3 src/gene_split.py ./modified_data ./result/gene_fna fna


/home/liuji/project/evolution-tutorials


In [46]:
# 使用mafft完成比对
! mkdir -p result/phylo_fna_gene
! ls result/gene_fna/ | while read file;do mafft --thread 8 --adjustdirection --auto result/gene_fna/${file} > result/phylo_fna_gene/${file%.fna}_aln.fna;done

nthread = 8
nadd = 0
inputfile = infile
thresholdtorev = 0.000000
reflim = 5000
mode = a
generating a scoring matrix for nucleotide (dist=200) ... done

Step 1/2
1   

Step 2/2
 1 / 4 (8 threads)   
makedirectionlist (nuc) Version 7.526
alg=m, model=DNA200 (2), 1.53 (4.59), 0.37 (1.11), noshift, amax=0.0
8 thread(s)

directionfile = _direction
inputfile = infile
subalignment = 0
subalignmentoffset = 0
4 x 915 - 897 d
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.526
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
8 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
    0 / 4
done.

Progressive alignment ... 
STEP     3 /3 (thread    1) 
don

In [47]:
#使用trimal完成修剪，去除gap
! ls result/phylo_fna_gene/*_aln.fna | while read file; do trimal -in ${file} -out ${file%_aln.fna}_AlnTrimed.fna -gappyout;done


In [48]:
#使用iqtree完成，如果数据量过大，可以使用fasttree快速构建一个树文件，暂时代替
! ls result/phylo_fna_gene/*_AlnTrimed.fna | while read file;do mkdir -p ${file%_AlnTrimed.fna}_iqtreeout;done
! ls result/phylo_fna_gene/*_AlnTrimed.fna | while read file; do iqtree -s $file -nt 8 -m MFP -bb 1000 -alrt 1000 -pre ${file%_AlnTrimed.fna}_iqtreeout/$(basename ${i%_AlnTrimed.fna}.tree); done


IQ-TREE multicore version 2.3.4 COVID-edition for Linux x86 64-bit built Jun 18 2024
Developed by Bui Quang Minh, James Barbetti, Nguyen Lam Tung, Olga Chernomor,
Heiko Schmidt, Dominik Schrempf, Michael Woodhams, Ly Trong Nhan, Thomas Wong

Host:    RCNSZOR36402A (AVX512, FMA3, 7 GB RAM)
Command: iqtree -s result/phylo_fna_gene/100045at50557_AlnTrimed.fna -nt 8 -m MFP -bb 1000 -alrt 1000 -pre result/phylo_fna_gene/100045at50557_iqtreeout/.tree
Seed:    88676 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Sat Aug 31 21:08:42 2024
Kernel:  AVX+FMA - 8 threads (8 CPU cores detected)

Reading alignment file result/phylo_fna_gene/100045at50557_AlnTrimed.fna ... Fasta format detected
Reading fasta file: done in 0.00011664 secs using 98.59% CPU
Alignment most likely contains DNA/RNA sequences
Alignment has 4 sequences with 897 columns, 58 distinct patterns
15 parsimony-informative, 286 singleton sites, 596 constant sites
                        Gap/Ambiguity  Composition 

# 2、利用修剪后的基因比对序列建物种树

In [52]:
#先将Trimal后的序列提取出来
! mkdir -p result/Aln_fna
! cp -r result/phylo_fna_gene/*_AlnTrimed.fna result/Aln_fna/

In [54]:
# 按照物种划分序列，得到对应物种的全序列，注意，缺失的基因用gap替换
! python3 src/species.py result/Aln_fna/ result/species_fna/ fna

In [63]:
#  由于本身序列就是比对修剪过后的，可以直接用来进行建树
! cat result/species_fna/* > result/species.fna
! mkdir -p result/species_iqtreeout
! iqtree -s result/species.fna -nt 8 -m MFP -bb 1000 -alrt 1000 -pre result/species_iqtreeout/species.tree

IQ-TREE multicore version 2.3.4 COVID-edition for Linux x86 64-bit built Jun 18 2024
Developed by Bui Quang Minh, James Barbetti, Nguyen Lam Tung, Olga Chernomor,
Heiko Schmidt, Dominik Schrempf, Michael Woodhams, Ly Trong Nhan, Thomas Wong

Host:    RCNSZOR36402A (AVX512, FMA3, 7 GB RAM)
Command: iqtree -s result/species.fna -nt 8 -m MFP -bb 1000 -alrt 1000 -pre result/species_iqtreeout/species.tree
Seed:    17802 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Sun Sep  1 17:09:50 2024
Kernel:  AVX+FMA - 8 threads (8 CPU cores detected)

Reading alignment file result/species.fna ... Fasta format detected
Reading fasta file: done in 0.000917375 secs using 37.5% CPU
Alignment most likely contains DNA/RNA sequences
Alignment has 4 sequences with 11235 columns, 153 distinct patterns
139 parsimony-informative, 2643 singleton sites, 8453 constant sites
                        Gap/Ambiguity  Composition  p-value
Analyzing sequences: done in 0.00612641 secs using 99.98% CPU